In [1]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD, evaluate
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('/Users/zbin/Downloads/customer_spu.csv')
df.head()

,customer_id,spu_id,try_buy_something,already_buy_something,viewed,collection,reviewd
0,29999,301821,2,2,1,0,0
1,29999,362469,1,1,0,0,0
2,30122,307288,1,1,0,0,0
3,30134,307365,1,2,0,0,0
4,30134,307367,1,2,0,0,0


In [3]:
len(df)

3611722

In [4]:
df['rating'] = 0.2 * df['try_buy_something'] + 0.3 * df['already_buy_something'] + 0.1 * df['viewed'] + 0.2 * df['collection'] + df['reviewd']
df.head()

,customer_id,spu_id,try_buy_something,already_buy_something,viewed,collection,reviewd,rating
0,29999,301821,2,2,1,0,0,1.1
1,29999,362469,1,1,0,0,0,0.5
2,30122,307288,1,1,0,0,0,0.5
3,30134,307365,1,2,0,0,0,0.8
4,30134,307367,1,2,0,0,0,0.8


In [5]:
df.sort_values('rating', ascending=False).head()

,customer_id,spu_id,try_buy_something,already_buy_something,viewed,collection,reviewd,rating
117377,46117,360792,0,2,56228,0,2,5625.4
117380,46117,361970,0,1,7557,0,1,757.0
117385,46117,363678,0,1,1266,0,1,127.9
1332244,372170,355506,0,328,5,1,0,99.1
2165871,650572,358861,0,1,917,0,0,92.0


In [6]:
df = df[~df.customer_id.isin([288942, 318769, 544120, 520906, 544120, 369949, 48632, 369556, 476317, 657795, 280205])]
df['rating'].describe()

count    3.610684e+06
mean     3.898472e-01
std      3.045077e+00
min      1.000000e-01
25%      1.000000e-01
50%      3.000000e-01
75%      4.000000e-01
max      5.625400e+03
Name: rating, dtype: float64

In [7]:
df.sort_values('rating', ascending=False).head()

,customer_id,spu_id,try_buy_something,already_buy_something,viewed,collection,reviewd,rating
117377,46117,360792,0,2,56228,0,2,5625.4
117380,46117,361970,0,1,7557,0,1,757.0
117385,46117,363678,0,1,1266,0,1,127.9
1332244,372170,355506,0,328,5,1,0,99.1
2165871,650572,358861,0,1,917,0,0,92.0


In [8]:
df[['customer_id', 'spu_id', 'rating']][:100000].head()

,customer_id,spu_id,rating
0,29999,301821,1.1
1,29999,362469,0.5
2,30122,307288,0.5
3,30134,307365,0.8
4,30134,307367,0.8


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3610684 entries, 0 to 3611721
Data columns (total 8 columns):
customer_id              int64
spu_id                   int64
try_buy_something        int64
already_buy_something    int64
viewed                   int64
collection               int64
reviewd                  int64
rating                   float64
dtypes: float64(1), int64(7)
memory usage: 247.9 MB


In [10]:
reader = Reader()
trainData = df[['customer_id', 'spu_id', 'rating']][1000000:]

In [11]:
testset = df[['customer_id', 'spu_id', 'rating']][:1000000]
df.groupby(['customer_id']).size().sort_values(ascending=False).head(10)

customer_id
91426     10864
315347     4157
96505      2883
578245     2614
415749     2356
332374     2342
42669      2117
372170     2028
263615     1749
147175     1662
dtype: int64

In [12]:
train_df_147175 = trainData[(trainData['customer_id'] == 147175) & (trainData['rating'] >= 0.8)]
train_df_147175 = train_df_147175.set_index('spu_id')
train_df_147175.sort_values('rating', ascending=False).head()

,customer_id,rating
spu_id,,
303942,147175,2.0
304000,147175,1.2
356003,147175,1.2
359270,147175,1.0
332162,147175,0.9


In [13]:
test_df_147175 = testset[(testset['customer_id'] == 147175) & (testset['rating'] >= 0.8)]
test_df_147175 = test_df_147175.set_index('spu_id')
test_df_147175.sort_values('rating', ascending=False).head()

,customer_id,rating
spu_id,,
307275,147175,19.9
304141,147175,19.0
303872,147175,17.4
304149,147175,16.0
303784,147175,15.7


测试集和训练集的数据差距很大

In [14]:
test_df_147175 = df['spu_id'].unique()
test_df_147175[:10]

array([301821, 362469, 307288, 307365, 307367, 307368, 350109, 362185,
       367245, 306248])

In [15]:
test_df_147175 = pd.DataFrame(test_df_147175, columns=['spu_id'])
test_df_147175.head()

,spu_id
0,301821
1,362469
2,307288
3,307365
4,307367


In [16]:
data = Dataset.load_from_df(trainData, reader)
data.split(n_folds=3)

svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])
trainset = data.build_full_trainset()
svd.fit(trainset)

/usr/local/var/pyenv/versions/3.6.0b1/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/usr/local/var/pyenv/versions/3.6.0b1/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.8232
MAE:  0.7494
------------
Fold 2
RMSE: 0.8280
MAE:  0.7494
------------
Fold 3
RMSE: 0.8252
MAE:  0.7496
------------
------------
Mean RMSE: 0.8255
Mean MAE : 0.7495
------------
------------


KeyboardInterrupt: 

In [ ]:
test_df_147175['Estimate_Score'] = test_df_147175['spu_id'].apply(lambda x: svd.predict(9, x).est)
test_df_147175.sort_values('Estimate_Score', ascending=False).head(20)


In [ ]:
test_df_147175.spu_id.head(20).values.tolist()

In [ ]:
test_df_147175[test_df_147175['spu_id'] == 307414]

In [ ]:
len(df[['customer_id', 'spu_id', 'rating']][1000000:])

通常用于推荐引擎评价的指标是称为最小均方根误差（Root Mean Squared Error，RMSE）的指标，它首先计算均方误差的平均值然后取其平方根。如果评级在1星到5星这个范围内，而我们得到的RMSE为1.0，那么就意味着我们的预测值和用户给出的真实评价相差了一个星级。

重新对输入数据进行处理
***去掉特殊用户数据
***去掉爆款产品的数据
***在有销量的产品中汇总view 收藏数据

加入评价